In [1]:
import argparse
import base64
import io
import logging
from typing import Dict

#import kfserving
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image

#logging.basicConfig(level=kfserving.constants.KFSERVING_LOGLEVEL)

In [2]:
transform = transforms.Compose([
    transforms.Resize(32),
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


def image_transform(instance):
    byte_array = base64.b64decode(instance['image']['b64'])
    image = Image.open(io.BytesIO(byte_array))
    im = Image.fromarray(np.asarray(image))
    res = transform(im)
    logging.info(res)
    return res.tolist()


CLASSES = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


def top_5(prediction):
    pred = torch.as_tensor(prediction)
    scores = torch.nn.functional.softmax(pred, dim=0)

    _, top_5 = torch.topk(pred, 5)

    results = {}
    for idx in top_5:
        results[CLASSES[idx]] = scores[idx].item()

    return results

In [3]:
with open('./cat.jpg', mode='rb') as file:
    img = file.read()
string = base64.encodebytes(img).decode("utf-8")
res = {'instances':[{'image': {'b64' : string}}]}

In [6]:
def preprocess(inputs: Dict) -> Dict:
        return {'instances': [image_transform(instance) for instance in inputs['instances']]}

In [13]:
img = base64.b64decode(res['instances'][0]['image']['b64'])
img = Image.open(io.BytesIO(img))
img = Image.fromarray(np.asarray(img))

In [17]:
transform(img).tolist()

[[[0.38823533058166504,
   0.23921573162078857,
   0.301960825920105,
   0.32549023628234863,
   0.24705886840820312,
   0.21568632125854492,
   0.20000004768371582,
   0.20000004768371582,
   0.21568632125854492,
   0.529411792755127,
   0.458823561668396,
   -0.019607841968536377,
   0.14509809017181396,
   0.2627451419830322,
   0.30980396270751953,
   0.27843141555786133,
   0.3647059202194214,
   0.46666669845581055,
   0.4745098352432251,
   0.45098042488098145,
   0.4117647409439087,
   0.2705882787704468,
   0.12156867980957031,
   0.09019613265991211,
   0.027451038360595703,
   -0.027450978755950928,
   -0.019607841968536377,
   0.058823585510253906,
   0.30980396270751953,
   0.45098042488098145,
   0.45098042488098145,
   0.45098042488098145],
  [0.45098042488098145,
   0.4745098352432251,
   0.23921573162078857,
   -0.027450978755950928,
   -0.05098038911819458,
   -0.08235293626785278,
   -0.05098038911819458,
   0.043137311935424805,
   0.23921573162078857,
   0.61568629

In [5]:
class ImageTransformer(kfserving.KFModel):
    def __init__(self, name: str, predictor_host: str):
        super().__init__(name)
        self.predictor_host = predictor_host

    def preprocess(self, inputs: Dict) -> Dict:
        return {'instances': [image_transform(instance) for instance in inputs['instances']]}

    def postprocess(self, inputs: Dict) -> Dict:
        return {'predictions': [top_5(prediction) for prediction in inputs['predictions']]}


if __name__ == "__main__":
    DEFAULT_MODEL_NAME = "model"

    parser = argparse.ArgumentParser(parents=[kfserving.kfserver.parser])
    parser.add_argument('--model_name', default=DEFAULT_MODEL_NAME,
                        help='The name that the model is served under.')
    parser.add_argument('--predictor_host', help='The URL for the model predict function', required=True)

    args, _ = parser.parse_known_args()

    transformer = ImageTransformer(args.model_name, predictor_host=args.predictor_host)
    kfserver = kfserving.KFServer()
    kfserver.start(models=[transformer])

NameError: name 'kfserving' is not defined